In [103]:
import numpy as np
import pandas as pd

In [2]:
!pip install PyPDF2
!pip install gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00


In [3]:
import os
from PyPDF2 import PdfReader

In [8]:
import gdown

# Replacing with Google Drive file ID
google_drive_file_id = '19h7t3xVNBS5KR9o2sLMh3i2uaje7ww7n'
output_filename = 'downloaded_pdf.pdf'

gdown.download(f'https://drive.google.com/uc?id={google_drive_file_id}', output_filename, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=19h7t3xVNBS5KR9o2sLMh3i2uaje7ww7n
To: /content/downloaded_pdf.pdf
100%|██████████| 1.26M/1.26M [00:00<00:00, 126MB/s]


'downloaded_pdf.pdf'

Now, creating a function to accept a single file path and extract text from it.

In [41]:
def extract_text_from_pdfs(file_path):

    if file_path.endswith('.pdf') and os.path.exists(file_path):
        reader = PdfReader(file_path)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    else:
        print(f"Error: File not found or not a PDF at {file_path}")
    return text

Now we can use the function with the downloaded file path to analyse the pdf file.

In [42]:
all_text = extract_text_from_pdfs(output_filename)

In [43]:
len(all_text)

83621

Now we need to make chunks for retrieval process

N.B: ekhane split kora hoyeche character onujaie. aro onek splitter ache jemon

How to: recursively split text
How to: split HTML
How to: split by character
How to: split code
How to: split Markdown by headers
How to: recursively split JSON
How to: split text into semantic chunks
How to: split by tokens

In [55]:
# as a text splitter we are using CharacterTextSplitter from Langchain framework
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [64]:
def split_docs(docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = text_splitter.create_documents([docs])
    return chunks

In [65]:
# testing the function
all_chunks = split_docs(all_text)

In [97]:
all_chunks[0].page_content

"অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জি জ্ঞাাসা ,\nঅপরির িতাআ ল া িয র িষ য়িাাং া\n১ম পত্র১। অ ন ুপ ল ে ি িাি া কী কল ি জ ীরিক া রন িবাহ  কি ল ত ন ?\nক) ডাক্তার্ি খ) ওকালর্ত গ) মাস্টার্ি ঘ) ব্যব্সা\n২। ে াে াল ক ভাগ্ য দ েি ত াি প্রধান এ ল জ ন্ট ি াি কাি ণ , ত াি-\nক) প্রর্তপজি খ) প্রভাব্  গ) র্ব্চক্ষণতা ঘ) কূট ব্ুর্ি\nর্নকচি অনুকেদটি পক়ে ৩ ও ৪ সংখযক প্রকেি উিি দাও।\nর্পতৃহীন দীপুি চাচাই র্িকলন পর্িব্াকিি কতিা। দীপু র্িজক্ষত হকলও তাি র্সিান্ত যনও া ি ক্ষমতা র্িল না। চাচা \nতাি র্ব্ক ি উকদযাগ র্নকলও যেৌতুক র্নক  ব্া়োব্ার়্ে কিাি কািকণ কনযাি র্পতা অপমার্ন ত যব্াধ ককি র্ব্ক ি \nআকলাচনা যভকে যদন। দীপু যমক টিি ির্ব্ যদকখ মুগ্ধ হকলও তাি চাচাকক র্কিুই ব্লকত পাকি নর্ন।\n৩। েীপুি ি াি াি সলে  ‘ অপ রিরি তা ' গ্ ল েি দ কান ি রিল েি রে   আ ল ে ?\nক) হর্িকিি খ) মামাি গ) র্িক্ষককি ঘ) র্ব্নুি\n৪। উক্ত ি রিল ে প্রাধানয  দ প ল য়লে  -\ni) যদৌিাত্ম ii) হীনম্মনযতা    iii) যলাভ\nর্নকচি যকানটি ঠিক ?\nক। i ও ii  খ। ii ও iii  গ। i ও iii  ঘ। i, ii ও iii\n৫. অ ন ুপ ল ে ি িয়স কত িে ি ?\nক) পঁর্চি   খ) িাব্বিি   গ

In [68]:
len(all_chunks)

95

**Next Step Embedding**

In [75]:
#!pip install langchain_openai

In [76]:
from langchain_openai import OpenAIEmbeddings

In [88]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

WE Need a vector store to store the embedding for future search. There are several vector stores like Chroma, FAISS, Lance. WE are using FAIS

In [86]:
#!pip install faiss-cpu
#!pip install langchain_community

In [89]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(all_chunks, embeddings)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

**It seems OpenAI is limiting our token number due to the free API call**
Let's try a new embedding system

In [90]:
from sentence_transformers import SentenceTransformer

In [98]:
embed_model = SentenceTransformer("intfloat/multilingual-e5-base")  # Supports Bengali & English


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [99]:
def embed_chunks(chunks):
    texts = [chunk.page_content for chunk in chunks]
    embeddings = embed_model.encode(texts, convert_to_tensor=True)
    return embeddings

In [100]:
embedded_chunks = embed_chunks(all_chunks)

In [102]:
embedded_chunks

tensor([[ 0.0213,  0.0548, -0.0090,  ..., -0.0231, -0.0521,  0.0311],
        [ 0.0189,  0.0433,  0.0016,  ..., -0.0228, -0.0359,  0.0412],
        [ 0.0209,  0.0426,  0.0009,  ..., -0.0365, -0.0353,  0.0473],
        ...,
        [ 0.0064,  0.0513, -0.0053,  ..., -0.0310, -0.0302,  0.0482],
        [ 0.0102,  0.0436, -0.0132,  ..., -0.0247, -0.0174,  0.0617],
        [-0.0174,  0.0366, -0.0039,  ..., -0.0363, -0.0347,  0.0525]])

**RETRIEVE**

Using FAISS for fast semantic search.

In [107]:
import faiss

def build_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings.cpu().detach().numpy())
    return index


In [109]:
#checking the function
index = build_faiss_index(embedded_chunks)

In [110]:
index.cached_l2norms

<faiss.swigfaiss_avx2.Float32Vector; proxy of <Swig Object of type 'std::vector< float > *' at 0x79ab61f8d1a0> >

Building the Retrieval Function

In [111]:
def retrieve_top_k(query, embed_model, chunks, embeddings, index, k=3):
    query_embedding = embed_model.encode([query], convert_to_tensor=True).cpu().detach().numpy()
    distances, indices = index.search(query_embedding, k)
    results = [chunks[i] for i in indices[0]]
    return results


In [165]:
#checking
query = "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?"
retrieved_chunks = retrieve_top_k(query, embed_model, all_chunks, embedded_chunks, index, k=3)

**RAG: Generating Answer**

In [125]:
#using transformer model from hugging face
from transformers import pipeline
from langchain_openai.chat_models import ChatOpenAI

In [132]:
#qa_model = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", device=0)
#apatoto onno model use kori, transformer onek boro
qa_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)
#not working

There are other models like

We have to use a multilingual model like mistralai/Mistral-7B-Instruct or google/gemma via Hugging Face Transformers, or OpenAI's gpt-4 or gpt-3.5. Because we are dealing with en and bn

In [122]:
# generate answer funtion
def generate_answer(query, retrieved_chunks):
    context = "\n\n".join([chunk.page_content for chunk in retrieved_chunks])
    prompt = f"Answer the following question using the provided context.\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"
    result = qa_model(prompt, max_new_tokens=200, do_sample=False)
    return result[0]['generated_text']

In [119]:
## RAG Pipeline
def rag_pipeline(query, docs_folder):
    docs = extract_text_from_pdfs(docs_folder)
    chunks = split_docs(docs)
    embeddings = embed_chunks(chunks)
    index = build_faiss_index(embeddings)
    top_chunks = retrieve_top_k(query, embed_model, chunks, embeddings, index)
    answer = generate_answer(query, top_chunks)
    return answer

In [120]:
query = "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?"
file_path = output_filename

In [123]:
rag_pipeline(query, file_path)

KeyboardInterrupt: 

**Just cheking**

In [142]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

Enter API key for Google Gemini: ··········


In [141]:
!pip install -qU "langchain[google-genai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [166]:
#checking manually
context = "\n\n".join([chunk.page_content for chunk in retrieved_chunks])

In [167]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate(
    template = """
    You are a helpful assistant.
    Answer only from the provided context.
    {context}
    Question: {question}
    """,
    input_variables = ["context", "question"]
)

In [168]:
final_prompt = prompt.invoke({"context": context, "question": query})

In [169]:
final_prompt

StringPromptValue(text="\n    You are a helpful assistant.\n    Answer only from the provided context.\n    i. প্রর্তব্াদ  \nii. যের্ণবব্ষময  \niii. ধমী  উৎসব্  োত্রা \nর্নকচি  যকানটি  সঠিক ? \n(ক) i, ii  (খ) i, iii  (গ) ii, iii  (ঘ) i, ii, iii       উিি : ক\nর্নকচিউদ্দীপকটিপক়ে৪৪ও৪৫নম্বিপ্রকেিউিিদাও:\nব্াংলাকদকিি অকনকপর্িব্ািযেৌতুককিিনযপুত্রব্ধূককর্নেিাতনককি।এমনইর্নেিাতকনির্িকািমমতা।\nমমতাতািস্বামীওস্বামীিপর্িব্াকিিসকলককযব্াঝাকনািযচষ্ট্াককির্কন্তুস্বামীিপর্িব্াকিিযলাকিনযতা\nদূকিিকথাতািস্বামীইর্কিুব্ুঝকতচা না।তাইমমতাব্াধযহক স্বামী-সংসািতযাগককিসিকার্িপ্রাথর্মক\nর্ব্দযালক র্িক্ষকর্হকসকব্চাকর্িগ্রহণককি। [ রস. দ িা.' ১৭]\n৪৪। উদ্দীপককিমমতাযতামািপঠিতযকানচর্িকত্রিপ্রর্তর্নর্ধত্বককি?\n(ক) মার্স  (খ) র্পর্স  (গ) কলযাণী   (ঘ) আহ্লার্দ         উিি : গ\n৪৫। প্রর্তর্নর্ধকত্বি  কািণ - \ni. প্রর্তব্াদী  মানর্সকতা  \nii. যপিাগত  িীব্ন  \niii. দব্ব্ার্হক  অব্িা  \nর্নকচি  যকানটি  সঠিক ? \n(ক) i, ii  (খ) i, iii  (গ) ii, iii  (ঘ) i, ii, iii       উিি : কিযাখ্যা : কািণকলযাণীওযস্টিনমাস্টাকিিকথািপ্রর্তব্া

In [170]:
result = model.invoke(final_prompt)

In [171]:
result.content

'এই প্রশ্নের উত্তর এই অংশে নেই।'